<a href="https://colab.research.google.com/github/dmlc/gluon-cv/blob/onnx/scripts/onnx/notebooks/classification/vgg13_bn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install --upgrade onnxruntime

In [ ]:

import numpy as np
import onnxruntime as rt
import urllib.request
import os.path 
from PIL import Image
    

In [ ]:

def fetch_model():
    if not os.path.isfile("vgg13_bn.onnx"):
        urllib.request.urlretrieve("https://apache-mxnet.s3-us-west-2.amazonaws.com/onnx/models/gluoncv-vgg13_bn-b08d2765.onnx", filename="vgg13_bn.onnx")
    return "vgg13_bn.onnx"

def prepare_img(img_path, input_shape):
    # input_shape: BHWC
    height, width = input_shape[1], input_shape[2]
    img = Image.open(img_path).convert('RGB')
    img = img.resize((width, height))
    img = np.asarray(img)
    img = np.expand_dims(img, axis=0).astype('float32')

    return img
    

**Make sure to replace the image you want to use**

In [ ]:

model = fetch_model()
img_path = "Your image"
img = prepare_img(img_path, (1, 224, 224, 3))
    

In [ ]:

# Create a onnx inference session and get the input name
onnx_session = rt.InferenceSession(model, None)
input_name = onnx_session.get_inputs()[0].name
# Make prediction
pred = onnx_session.run([], {input_name: img})[0]
    


(Optional) We use mxnet and gluoncv to process the result.

Feel free to process the result your own way


In [ ]:
!pip3 install --upgrade mxnet gluoncv

In [ ]:

def prepare_label():
    attrib = ImageNet1kAttr()
    return attrib.classes

import mxnet as mx
from gluoncv.data import ImageNet1kAttr

labels = prepare_label()    
pred = mx.nd.array(pred)
prob = mx.nd.softmax(pred)[0].asnumpy()
# find the 5 class indices with the highest score
ind = mx.nd.topk(pred, k=5)[0].astype('int').asnumpy().tolist()
# print the class name and predicted probability
print('The input picture is classified to be')
for i in range(5):
    print('- [%s], with probability %.3f.'%(labels[ind[i]], prob[ind[i]]))
    